# pickleから辞書を取得

In [1]:
# # （tsv形式の辞書からpickleを取得）

# with open('data_extract/ontology.tsv', mode='r', newline='', encoding='utf-8') as f:
#     tsv_reader = csv.reader(f, delimiter='\t')
#     data = [row for row in tsv_reader if row != []]

# new_data = []
# key = data[0]

# for d in data[1:]:
#     data_d = {}
#     for i in range(len(key)):
#         data_d[key[i]] = d[i]
#     new_data.append(data_d)

# import pickle
# with open("data_extract/ontology.pickle", 'wb') as f:
#     pickle.dump(new_data,f)

import pickle
with open("data_extract/ontology.pickle", 'rb') as f:
    data = pickle.load(f)

# DeepL API準備

In [57]:
with open('data_secret/DeepL_api.txt', 'r', encoding='UTF-8') as f:
    k = f.read()

In [58]:
import requests

def ENtoJA(text):
    # URLクエリに仕込むパラメータの辞書を作っておく
    params = {"auth_key": k,
                "text": text,
                "source_lang": 'EN', # 入力テキストの言語を英語に設定
                "target_lang": 'JA'  # 出力テキストの言語を日本語に設定（JPではなくJAなので注意）
             }

    # パラメータと一緒にPOSTする
    request = requests.post("https://api-free.deepl.com/v2/translate", data=params) # free用のURL、有料版はURLが異なります
    result = request.json()
    return result["translations"][0]["text"]

In [65]:
for d in data:
    if "trans_name" in d:
        pass
    else:
        d["trans_name"] = ENtoJA(d["en_name"])
        print('\r%s                                            ' %(d["trans_name"]), end='')

In [72]:
import pickle
with open("DeepL_11057in50640.bin", 'wb') as f:
    pickle.dump(data,f)
    
# 翻訳結果
# ・50640項目中11057の地点でAPIの利用制限到達，翻訳精度微妙
# ・英語を直に日本語読みとかのほうがまだいいかも
# ・作成したデータは上記のバイナリファイルに保存してtrushに移動済み

In [117]:
import pickle
with open("trush/DeepL_11057in50640.bin", 'rb') as f:
    DeepL_data = pickle.load(f)

EOFError: Ran out of input

# ローマ字読み付与

In [76]:
# !pip install pykakasi

In [77]:
import pykakasi

kakasi = pykakasi.kakasi() # インスタンスの作成
kakasi.setMode('H', 'a') # ひらがなをローマ字に変換するように設定
kakasi.setMode('K', 'a') # カタカナをローマ字に変換するように設定
kakasi.setMode('J', 'a') # 漢字をローマ字に変換するように設定
conversion = kakasi.getConverter() # 上記モード設定の適用


for d in data:
    if "romaji_name" in d:
        pass
    else:
        d["romaji_name"] = conversion.do(d["ja_name"])
        print('\r%s                                            ' %(d["romaji_name"]), end='')

<ipython-input-77-199bc52e4aae>:4: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi.setMode('H', 'a') # ひらがなをローマ字に変換するように設定
<ipython-input-77-199bc52e4aae>:5: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi.setMode('K', 'a') # カタカナをローマ字に変換するように設定
<ipython-input-77-199bc52e4aae>:6: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi.setMode('J', 'a') # 漢字をローマ字に変換するように設定
<ipython-input-77-199bc52e4aae>:7: DeprecationWarning: Call to deprecated method getConverter. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  conversion = kakasi.getConverter() # 上記モード設定の適用
<ipython-input-77-199bc52e4aae>:14: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  d["romaji_na

oodarumainko                                                                           

# ネットワーク作成　（&親子tsv作成)

In [137]:
!pip install networkx

You should consider upgrading via the 'c:\users\teramoto\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
child_parent_data = []
for d in data:
    child_parent_data.append({"id":d["id"],"parent_taxon":d["parent_taxon"]})

import csv
with open('data_extract/child_parent.tsv','w',encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = ["id","parent_taxon"],delimiter = "\t")
    writer.writeheader()
    writer.writerows(child_parent_data)

In [24]:
import networkx as nx
G = nx.DiGraph() #無向グラフを作成 

In [25]:
for d in child_parent_data:
    G.add_edge(d["parent_taxon"],d["id"])

In [28]:
# !pip install --upgrade matplotlib

In [31]:
from matplotlib import pyplot as plt

nx.draw(G, with_labels = True)
plt.show()

#描画試みの結果
# ・データが大きすぎるため私の個人PCでは不可能

# パスデータの抜け確認

#### 言語によるパスおよびそのランク表現を追加

In [33]:
dict_data = {}
for d in data:
    dict_data[d["id"]]=d

In [ ]:
for d in data:
    if "path_name" in d:
        pass
    else:
        path_list = d["path"].replace("'","")
        d["path_name"] = conversion.do(d["ja_name"])
        print('\r%s                                            ' %(d["romaji_name"]), end='')

In [81]:
for d in data:
    path_list = d["path"].replace("'/","").replace("'","").split("/")
    path_list

    ja_path_name = []
    en_path_name = []
    path_taxon_rank_name = []

    for path in path_list:
        if path in dict_data:
            if dict_data[path]["ja_name"] != "":
                ja_path_name.append(dict_data[path]["ja_name"])
                en_path_name.append(dict_data[path]["en_name"])
            else:
                ja_path_name.append(dict_data[path]["en_name"])
                en_path_name.append(dict_data[path]["en_name"])

            if dict_data[path]["taxon_rank_name"] != "":
                path_taxon_rank_name.append(dict_data[path]["taxon_rank_name"])
        else:
            ja_path_name.append(path)
            path_taxon_rank_name.append("Unknown")

    d["path"] = "/".join(path_list)
    d["ja_path_name"] = "/".join(ja_path_name)
    d["en_path_name"] = "/".join(en_path_name)
    d["path_taxon_rank_name"] = "/".join(path_taxon_rank_name)

In [84]:
filedname_list = ['id','en_name','ja_name','en_aliases','ja_aliases','img_urls','taxon_name','taxon_rank','taxon_rank_name','taxon_rank_ja_name','parent_taxon','parent_taxon_name','parent_taxon_ja_name',"path","ja_path_name","en_path_name","path_taxon_rank_name"]

import csv

with open('data_extract/ontology_path.tsv','w',encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = filedname_list,delimiter = "\t")
    writer.writeheader()
    writer.writerows(data)
    
with open("data_extract/ontology_path.pickle", 'wb') as f:
    pickle.dump(data,f)

#### パスのランク種を集計し，完全版パスを人手で復元

In [86]:
path_set = set()
for d in data:
    path_set.add(d["path_taxon_rank_name"])
    
path_list = list(path_set)
path_list.sort()

with open('data_extract/path_category.txt', 'w') as f:
    for row in path_list:
        f.write(row)
        f.write("\n")

#完全な場合のパス
complete_path = ["subclass","infraclass","superorder","order","suborder","infraorder","superfamily","family","subfamily","tribe","genus","subgenus","species","subspecies","form"]

# すべて人力で修正していくと持たないので，泣き声データにあるもののみ人手で修正することに

In [119]:
data[0]

{'id': 'Q1001586',
 'en_name': "Finsch's Francolin",
 'ja_name': 'チャバネシャコ',
 'en_aliases': 'Francolinus finschi,Scleroptila finschi',
 'ja_aliases': '',
 'img_urls': '',
 'taxon_name': 'Scleroptila finschi',
 'taxon_rank': 'Q7432',
 'taxon_rank_name': 'species',
 'taxon_rank_ja_name': '種',
 'parent_taxon': 'Q1286431',
 'parent_taxon_name': 'Scleroptila',
 'parent_taxon_ja_name': '',
 'path': 'Q1286431/Q1001586',
 'ja_path_name': 'Scleroptila/チャバネシャコ',
 'en_path_name': "Scleroptila/Finsch's Francolin",
 'path_taxon_rank_name': 'genus/species'}

In [ ]:
# 模範